### CARACTÉRISTIQUES DE CE TRAIN : 

- On rajoute de la batch normalisation
- On le fait sur une partie des images denoised pour voir ce que ça donne

In [4]:
pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 126.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 10.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 65.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 83.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 88.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 88.9 MB/s eta 0:00:00:00

In [ ]:
pip install librosa

In [5]:
import librosa
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import multiprocessing as mp
from tqdm.notebook import tqdm

2024-05-27 11:04:37.919159: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 11:04:37.924352: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 11:04:37.998259: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 11:04:40.480524: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# Fonctions pour load les audio puis en créer des images

def load_audio(file_path, sr=22050):
    audio, sr = librosa.load(file_path, sr=sr)
    return audio, sr

def get_spectrogram(audio, sr=22050, n_mels=128, fmax=8000):
    spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels, fmax=fmax)
    return spectrogram

def generate_spectrogram(row, audio_dir, audio_len=22050*5, n_mels=128, fmax=8000, fixed_shape=(128, 128)):
    file_path = os.path.join(audio_dir, row['filename'])
    audio, sr = load_audio(file_path)
    audio = audio[:audio_len]
    spec = get_spectrogram(audio, sr=sr, n_mels=n_mels, fmax=fmax)
    spec_db = librosa.power_to_db(spec, ref=np.max)

    # Ensure the spectrogram has the fixed shape
    if spec_db.shape[1] < fixed_shape[1]:
        padding = fixed_shape[1] - spec_db.shape[1]
        spec_db = np.pad(spec_db, ((0, 0), (0, padding)), mode='constant')
    else:
        spec_db = spec_db[:, :fixed_shape[1]]

    return spec_db

def preprocess_metadata(df, meta_columns):
    for col in meta_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df.dropna(subset=meta_columns)  # Drop rows with NaN values in meta columns
    return df

In [ ]:
df_meta = pd.read_csv("train_metadata.csv")
